# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [17]:
from cmath import nan
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import numpy as np

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)
                
    def __truediv__(self, other: Union["Dual", Number])-> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / o_value / o_value)
            case Number():
                if other == 0:
                    print("Division by 0, reurn nan")
                    return Dual(nan,nan)
                return Dual(self.value / float(other),  self.d  / float(other))
            
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __neg__(self)-> "Dual":
        return (Dual(0, 0) - self)
    
    def __pos__(self)-> "Dual":
        return (Dual(0, 0) + self)
    
    def __pow__(self, other: Number)-> "Dual":
        if isinstance(other, Number):
            return Dual(self.value ** other, float(other) * (self.d) * (self.value) ** (float(other) - 1))
        else:
            raise TypeError("power for numbers only")
    

        

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 
    def __rsub__(self, other: Union["Dual", Number]):
        return -self.__sub__(other)
    def __rtruediv__(self, other: Number):
        return other * self ** -1
#functions overload
def cos(x: "Dual")-> "Dual":
    match x:
        case Dual(value, d):
            return Dual(np.cos(value),-np.sin(value)*d)
        case Number():
            return Dual(np.cos(x),0.)
    #return Dual(np.cos(x.value),-np.sin(x.value)*x.d)   
    
def sin(x: "Dual")-> "Dual":
    match x:
        case Dual(value, d):
            return Dual(np.sin(value),np.cos(value)*d)

        case Number():
            return Dual(np.cos(x),0.)

def exp(x: "Dual")-> "Dual":
    match x:
        case Dual(value, d):
            return Dual(np.exp(value),np.exp(value)*d)
        case Number():
            return Dual(np.exp(x),0.)

def log(x: "Dual")-> "Dual":
    match x:
        case Dual(value, d):
            if(value == 0.):
                print("division by 0 error, return nan")
                return Dual(nan,nan)
            return Dual(np.log(value),d/value)
        case Number():
            if x <= 0.:
                print("Wrong argument, return nan")
                return Dual(nan,nan)
            return Dual(np.log(x),0)

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d
 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [18]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x ** 3 - 1

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

60.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

Недостаток: необходимо использовать конструкцию match union / number довольно часто для каждой операции, когда можно константу const в функции заменять на Dual(const, 0) в функции diff и избежать этого

In [19]:
# symmetry test
def f_symm(x: float) -> float:
    return 5 * x * x * x

def f_symm2(x: float) -> float:
    return 5 * -x * -x * x

def f_pow(x: float) -> float:
    return 5 * x ** (-2)


f_diff = diff(f_symm)
f_pow_diff = diff(f_pow)
f_diff2 = diff(f_symm2)

# значение производной в точке x = 2
assert f_diff(2) == f_diff(-2) == 60 == f_diff2(2), 'Symmetry test failed'
assert f_pow_diff(5) == -0.08, 'Power test error'

In [20]:
# asymmetry test and constants
@diff
def f_symm1(x: float) -> float:
    return 5 * x * x * x + 123412
@diff
def f_symm2(x: float) -> float:
    return 5 * x * x * -x + 123412
@diff
def f_asymm(x: float) -> float:
    return 5 * x * -x * x * x - 123124
@diff
def f_truediv(x: float) -> float:
    return (5 * x * -x * x - 123124)/2

# значение производной в точке x = 2

assert f_symm1(2) == f_symm1(-2) == -f_symm2(-2) == 60, 'Symmetry test failed'
assert f_asymm(-2) == 160 == -f_asymm(2), 'Assymetry test failed'
assert -30 == f_truediv(2), 'Truediv test failed'

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [21]:
# ваш код   
@diff
def f_sin(x: float) -> float:
    return 5 * sin(x)
@diff
def f_cos(x: float) -> float:
    return 5 * cos(x)
@diff
def f_exp(x: float) -> float:
    return 5 * exp(-x/2)
@diff
def f_log(x: float) -> float:
    return 5 * log(x/2)
@diff
def complex_func(x: float) -> float:
    return 5 * sin(cos(x))
assert f_sin(2) == 5 * np.cos(2), 'Sin test failed'
assert f_cos(2) == -5 * np.sin(2), 'Cos test failed'
assert f_exp(2) == -5/2 * np.exp(-1), 'Exp test failed'       
assert f_log(4) == 5/4, 'Log test failed'
assert round(complex_func(12),8) == round((-5 * np.sin(12) * np.cos(np.cos(12))),8) , 'Complex function test failed'

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [22]:
from scipy.misc import derivative

def f1(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f2(x: float) -> float:
    return (5 * x * x + 2 * x + 2) / 2

def f3(x: float) -> float:
    return 5 * x / x - 2 * x - 2

def complex_func_test(x: float) -> float:
    return 5 * np.sin(np.cos(x))

def complex_func_test2(x: float) -> float:
    return np.exp(x * np.cos(x))

@diff
def complex_func2(x: float) -> float:
    return exp(x * cos(x))
assert derivative(f1, 2.) == diff(f1)(2)
assert derivative(f2, -2.) == diff(f2)(-2)
assert derivative(f3, 2.) == diff(f3)(2)
assert round(derivative(complex_func_test, 12, dx=1e-6),5) == round(complex_func(12),5)

assert round(derivative(complex_func_test2, 10, dx=1e-6),5) == round(complex_func2(10),5), "Complex Function test failed"

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [32]:
# ваш код
from random import randint
dict_func_codes = {
    #argument
    0:"x",
    1:randint(-4,7 or 8),
    
    #functions
    2:"sin(",
    3:"cos(",
    4:"log(",
    5:"exp(",
    
    #two part opertaions
    6:"+",
    7:"/",
    8:"-",
    9:"*",
    10:"**"
}

def add_argument(func_str: str)-> str:
    argtype = randint(0, 1)
    if argtype == 0:
            func_str += "x"
    else:
        func_str += str(randint(-255,255-1)) or 255
    return func_str

def generate_func_str(operations_num: int):
    OPERATIONS_LEN = operations_num
    BRACKET_CLOSE_CHANCE = 0.4
    func_str = ""
    only_pos_input = True
    unmatching_brackets = 0
    unmatching_operations = False
    arg_type = 'start' #x, const,argtype 0, sin, cos, log - argtype 2, +,-,*,**,/,- - argtype 3

    #init
    code = randint(0,5)
    if(code < 2):
        arg_type = 'arg'
    else:
        arg_type = 'func'

    for i in range(OPERATIONS_LEN):
        dict_func_codes[1] = randint(-4,7) or 8
        if(dict_func_codes[1] == 0):
            print("impossible")
        if(arg_type == 'arg'):
            if(only_pos_input):
                dict_func_codes[1] = randint(1,7) or 8
            code = randint(0,1)
            unmatching_operations = False
            func_str += str(dict_func_codes[code])
            if(unmatching_brackets):
                close_code = randint(0,100)
                if(close_code < 100 * BRACKET_CLOSE_CHANCE):
                    func_str += ")"
                    only_pos_input = False
                    unmatching_brackets -= 1
            arg_type = 'operator'
            continue
        elif(arg_type == 'func'):
            code = randint(2,5)
            unmatching_operations = True
            if(code == 4):
                only_pos_input = True
            unmatching_brackets += 1
            arg_type = 'arg'
        elif(arg_type == 'operator'):
            code = randint(6,10)
            unmatching_operations = True
            if(code == 10): # reroll if power occur to decreace chance
                code = randint(6,10)
            if(code == 10):
                arg_type = 'const'
            else:
                arg_code = randint(0,3)
                if(arg_code != 0):
                    arg_type = 'arg'
                else:
                    arg_type = 'func'

        elif(arg_type == 'const'):
            if(code == 7): # avoid division by 0
                arg_type = 'const'
                dict_func_codes[1] = randint(1,32)
            code = 1
            unmatching_operations = False
            arg_type = 'operator'
        func_str += str(dict_func_codes[code])

    if(unmatching_operations):
        if(code == 10):
            func_str += str(randint(-4,7)) or 8
        else:
            code = randint(0,1)
            func_str += str(dict_func_codes[code])
    if(unmatching_brackets):
        func_str += unmatching_brackets * ")"
        unmatching_brackets = 0
    return func_str

In [34]:
def get_random_func_repr():
    func_str = generate_func_str(10)
    while (func_str.find('x') == -1):
        func_str = generate_func_str(10)
    test_func = eval("lambda x: " + func_str)

    test_np_str = func_str.replace("log","np.log").replace("sin","np.sin").replace(
                    "cos","np.cos").replace("exp","np.exp")
    test_np_func = eval("lambda x: " + test_np_str)
    return func_str, test_func, test_np_func

In [60]:
from decimal import DivisionByZero
from random import uniform


for idx in range (50):
    func_str, test_func, test_np_func = get_random_func_repr()
    point = uniform(1,5)
    print("Function ",idx," ",func_str)
    try:
        print("Function auto diff in point ", point," =",round(diff(test_func)(point),5))
        print("Function scipy diff in point ", point," =",round(derivative(test_np_func, point, dx=1e-6),5))
    except OverflowError:
        try:
            print("Recult too large, decreacing point value")
            print("Function auto diff in point ", point," =",round(diff(test_func)(1/point),5))
            print("Function scipy diff in point ", point," =",round(derivative(test_np_func, 1/point, dx=1e-6),5))
        except OverflowError:
            print("Function overflow, rerolling...")
            func_str, test_func, test_np_func = get_random_func_repr()
            point = uniform(1,5)
            print("Function ",idx," ",func_str)
            print("Function auto diff in point ", point," =",round(diff(test_func)(point),5))
        print("Function scipy diff in point ", point," =",round(derivative(test_np_func, point, dx=1e-6),5))
    except DivisionByZero:
        print("Function auto diff in point ", point," =",round(diff(test_func)(point + 1),5))
        print("Function scipy diff in point ", point," =",round(derivative(test_np_func, point + 1, dx=1e-6),5))

Function  0   sin(x+x*5)*cos(6--2)
Function auto diff in point  2.304842755195411  = -0.26474
Function scipy diff in point  2.304842755195411  = -0.26474
Function  1   cos(x)*log(5)-x/-2*-4
Function auto diff in point  2.01621218950554  = -3.45241
Function scipy diff in point  2.01621218950554  = -3.45241
Function  2   sin(x/x)-log(3/exp(3))
Function auto diff in point  3.0759942088697847  = 0.0
Function scipy diff in point  3.0759942088697847  = 0.0
Function  3   sin(x)-1+x--1-exp(x)
Function auto diff in point  4.74554144873536  = -114.03695
Function scipy diff in point  4.74554144873536  = -114.03695
Function  4   exp(x-x)--2-5+sin(8)
Function auto diff in point  3.984916973470386  = 0.0
Function scipy diff in point  3.984916973470386  = 0.0
Function  5   cos(x/4)+x/x**5
Function auto diff in point  3.3328359628110684  = -0.19475
Function scipy diff in point  3.3328359628110684  = -0.19475
Function  6   sin(7+5**4+x/3)
Function auto diff in point  1.0555691747105325  = -0.20935
Func

C:\Users\dyush\AppData\Local\Temp\ipykernel_7124\1367983831.py:95: RuntimeWarning: invalid value encountered in log
  return Dual(np.log(value),d/value)
<string>:1: RuntimeWarning: invalid value encountered in log


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [26]:
# ваш код